In [1]:
%load_ext lab_black

In [2]:
import pandas as pd
import numpy as np

In [3]:
demand = pd.read_excel("opt_example.xlsx", "demand")
bundles = pd.read_excel("opt_example.xlsx", "bundles")

In [4]:
from pulp import LpVariable, LpInteger, value
from pulp import LpProblem, LpMinimize, LpStatus
from pulp import PULP_CBC_CMD

all_bundle_variables = []
all_missing_variables = []
types = ["b1", "b2", "b3", "b4", "b5"]
missing_weights = np.array([4] * 5)
for product, df in demand.groupby("product"):
    problem = LpProblem("allocation", LpMinimize)
    total_demand = df["demand"].sum()
    bundle_variables = {}
    missing_variables = {}
    for type_ in types:
        bundle_variables[type_] = LpVariable(name=type_, lowBound=0, cat=LpInteger)
        missing_variables[type_] = LpVariable(
            name=f"missing_{type_}", lowBound=0, cat=LpInteger
        )
    bundle_weight_matrix = df[types].fillna(0).to_numpy()
    bundle_weights = bundle_weight_matrix.sum(axis=0)
    bundle_vector = np.array(list(bundle_variables.values()))
    n_types = df.shape[0]
    missing_weight_matrix = np.tile(missing_weights, (n_types, 1))
    missing_vector = np.array(list(missing_variables.values()))

    bundle_exprs = bundle_vector * bundle_weight_matrix

    for qty, bundle in zip(df["demand"], bundle_exprs):
        problem += sum(bundle) + sum(missing_vector) >= qty
    total_bundles_expr = bundle_weights * bundle_vector
    total_missing_expr = missing_weights * missing_vector
    problem += sum(total_bundles_expr) <= 2 * total_demand

    OBJECTIVE = sum(total_bundles_expr) + sum(total_missing_expr)
    problem += OBJECTIVE
    status = problem.solve(PULP_CBC_CMD(msg=False))
    bundle_variables["product"] = product
    missing_variables["product"] = product
    all_bundle_variables.append(bundle_variables)
    all_missing_variables.append(missing_variables)
print(LpStatus[problem.status])
allocation = pd.DataFrame(all_bundle_variables)
missing = pd.DataFrame(all_missing_variables)

allocation[types] = allocation[types].applymap(value)
allocation["total"] = allocation[types].sum(axis=1)

missing[types] = missing[types].applymap(value)
missing["total"] = missing[types].sum(axis=1)

Optimal


In [5]:
print(problem)

allocation:
MINIMIZE
6.0*b1 + 6.0*b2 + 6.0*b3 + 6.0*b4 + 6.0*b5 + 4*missing_b1 + 4*missing_b2 + 4*missing_b3 + 4*missing_b4 + 4*missing_b5 + 0.0
SUBJECT TO
_C1: b1 + 3 b3 + 3 b4 + missing_b1 + missing_b2 + missing_b3 + missing_b4
 + missing_b5 >= 78

_C2: 2 b1 + 3 b2 + 4 b5 + missing_b1 + missing_b2 + missing_b3 + missing_b4
 + missing_b5 >= 96

_C3: b1 + 3 b3 + 2 b5 + missing_b1 + missing_b2 + missing_b3 + missing_b4
 + missing_b5 >= 63

_C4: 2 b1 + 3 b2 + 3 b4 + missing_b1 + missing_b2 + missing_b3 + missing_b4
 + missing_b5 >= 35

_C5: 6 b1 + 6 b2 + 6 b3 + 6 b4 + 6 b5 <= 544

VARIABLES
0 <= b1 Integer
0 <= b2 Integer
0 <= b3 Integer
0 <= b4 Integer
0 <= b5 Integer
0 <= missing_b1 Integer
0 <= missing_b2 Integer
0 <= missing_b3 Integer
0 <= missing_b4 Integer
0 <= missing_b5 Integer



In [6]:
allocation

,b1,b2,b3,b4,b5,product,total
0,2.0,29.0,9.0,0.0,0.0,casaco,40.0
1,2.0,18.0,13.0,0.0,0.0,chinelo,33.0
2,0.0,23.0,8.0,0.0,0.0,guarda-chuva,31.0
3,0.0,25.0,21.0,0.0,3.0,mp3,49.0
4,0.0,0.0,14.0,12.0,24.0,toalha,50.0


In [7]:
missing

,b1,b2,b3,b4,b5,product,total
0,1.0,0.0,0.0,0.0,0.0,casaco,1.0
1,1.0,0.0,0.0,0.0,0.0,chinelo,1.0
2,1.0,0.0,0.0,0.0,0.0,guarda-chuva,1.0
3,2.0,0.0,0.0,0.0,0.0,mp3,2.0
4,0.0,0.0,0.0,0.0,0.0,toalha,0.0


### criar uma tabela com a comparação da demanda versus alocado

In [8]:
# exemplo
pd.DataFrame(
    [
        ["casaco", "type A", 17, "<valor>"],
        ["casaco", "type C", 32, "<valor>"],
        ["guarda-chuva", "type A", 23, "<valor>"],
    ],
    columns=["product", "type", "demand", "allocated"],
)

,product,type,demand,allocated
0,casaco,type A,17,<valor>
1,casaco,type C,32,<valor>
2,guarda-chuva,type A,23,<valor>
